# Universidad Autónoma de Yucatán

## Facultad de Matemáticas

### Natural Language Processing

**Teacher:** Dr. Jorge Carlos Reyes

**Student:** Dayan Bravo Fraga

# BERT

## Bidirectional Encoder Representations from Transformers

# Practice: "Spam Detection"

## Download Corpus from GitHub (only for Colab)

In [ ]:
import sys

in_colab: bool = 'google.colab' in sys.modules
if in_colab:
    print('Is running in Colab')
    import os

    # Verify if the corpus is already downloaded
    if not os.path.isfile('spam.csv'):
        import gdown

        # Downloading the corpus
        print("Downloading Data")
        url_corpus = "https://raw.githubusercontent.com/dayan3847/natural_language_processing/dayan3847/task_final/spam.csv"
        gdown.download(url_corpus, quiet=False)

    # Verify if the model is already downloaded
    if not os.path.isdir('bert_model'):
        # Verify if the model ZIP is already downloaded
        if not os.path.isfile('bert_model.zip'):
            get_ipython().system('pip install mega.py')
            from mega import Mega

            url_model = 'https://mega.nz/file/U3hF2JYC#oqCtm3hgCYA-q8SQ-f5BL81tK_mCJaKbol8FyzNOK2g'
            Mega().download_url(url_model)
        # Unzip the model
        import zipfile

        with zipfile.ZipFile('bert_model.zip', 'r') as zip_ref:
            zip_ref.extractall()

else:
    print('Is not running in Colab')

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

2023-05-24 10:20:40.920548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 10:20:41.510333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Import Data

In [20]:
spam_data = pd.read_csv('spam.csv')
spam_data.head()

,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


## Perform Basic Analysis

In [21]:
spam_data.groupby('target').describe()

text                                                               
       count unique                                                top freq
target                                                                     
ham     4825   4516                             Sorry, I'll call later   30
spam     747    653  Please call our customer service representativ...    4

## Balancing dataset

In [23]:
spam_data_spam = spam_data[spam_data['target'] == 'spam']
spam_data_ham = spam_data[spam_data['target'] == 'ham']
df_ham_down_sampled = spam_data_ham.sample(n=len(spam_data_spam), random_state=0)
spam_data_balanced = pd.concat([df_ham_down_sampled, spam_data_spam])
spam_data_balanced.groupby('target').describe()

text                                                               
       count unique                                                top freq
target                                                                     
ham      747    736                             Sorry, I'll call later    5
spam     747    653  Please call our customer service representativ...    4

In [24]:
spam_data_original = spam_data
spam_data = spam_data_balanced
print(spam_data_original.groupby('target').describe())
print(spam_data.groupby('target').describe())

        text                                                               
       count unique                                                top freq
target                                                                     
ham     4825   4516                             Sorry, I'll call later   30
spam     747    653  Please call our customer service representativ...    4
        text                                                               
       count unique                                                top freq
target                                                                     
ham      747    736                             Sorry, I'll call later    5
spam     747    653  Please call our customer service representativ...    4


# Change Target to (0, 1)

In [26]:
spam_data['target'] = spam_data['target'].apply(lambda x: 1 if x == 'spam' else 0)
spam_data.head()

,text,target
3031,Jokin only lar... :-) depends on which phone m...,0
4932,Once free call me sir.,0
5060,"Sorry, I'll call you later. I am in meeting sir.",0
4008,Ha... Then we must walk to everywhere... Canno...,0
4230,U really pig leh sleep so much. My dad wake me...,0


In [27]:
spam_data.groupby('target').describe()

text                                                               
       count unique                                                top freq
target                                                                     
0        747    736                             Sorry, I'll call later    5
1        747    653  Please call our customer service representativ...    4

## Train Test Split

In [28]:
x_train, x_test, y_train, y_test = train_test_split(
    spam_data['text'],
    spam_data['target'],
    random_state=0,
    stratify=spam_data['target'],
)

## Downloading the BERT model

In [29]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2023-05-24 10:38:58.162293: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-24 10:38:58.162636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-24 10:38:58.162744: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-24 10:38:58.163049: I tensorflow/core/

## Initializing the BERT layers

In [30]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

2023-05-24 10:39:24.075764: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-24 10:39:24.091881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-05-24 10:39:24.190124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_2' with dtype int32 and shape [?,128]
	 [[{{node inputs_2}}]]
2023-05-

## Initializing the neural network layers

In [31]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [32]:
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

## Model compiling

In [34]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

## Fitting the model

In [35]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


2023-05-24 10:41:33.266928: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_3' with dtype float and shape [?,?,768]
	 [[{{node gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_3}}]]
2023-05-24 10:41:33.267012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_4' with dtype float and shape [?,?,768]
	 [[{{node gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_4}}]]
2023

35/35 [==============================] - 82s 2s/step - loss: 0.5766 - accuracy: 0.7286 - precision: 0.7294 - recall: 0.7268
Epoch 2/10
35/35 [==============================] - 74s 2s/step - loss: 0.4713 - accuracy: 0.8384 - precision: 0.8307 - recall: 0.8500
Epoch 3/10
35/35 [==============================] - 74s 2s/step - loss: 0.4034 - accuracy: 0.8527 - precision: 0.8388 - recall: 0.8732
Epoch 4/10
35/35 [==============================] - 76s 2s/step - loss: 0.3668 - accuracy: 0.8866 - precision: 0.8701 - recall: 0.9089
Epoch 5/10
35/35 [==============================] - 77s 2s/step - loss: 0.3437 - accuracy: 0.8786 - precision: 0.8655 - recall: 0.8964
Epoch 6/10
35/35 [==============================] - 77s 2s/step - loss: 0.3213 - accuracy: 0.8866 - precision: 0.8676 - recall: 0.9125
Epoch 7/10
35/35 [==============================] - 76s 2s/step - loss: 0.3129 - accuracy: 0.8875 - precision: 0.8767 - recall: 0.9018
Epoch 8/10
35/35 [==============================] - 73s 2s/step - 

## Saving trained model

In [36]:
model.save('bert_model')

2023-05-24 10:54:18.275450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-24 10:54:18.292981: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'text' with dtype string and shape [?]
	 [[{{node text}}]]
2023-05-24 10:54:18.365493: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_2' with dtype int32 and shape [?,128]
	 [[{{node inputs_2}}]]
2023-05-24 10:54:18.36

INFO:tensorflow:Assets written to: bert_model/assets


INFO:tensorflow:Assets written to: bert_model/assets


## Loading trained model

In [24]:
model = tf.keras.models.load_model('bert_model')

## Customize prediction function

In [49]:
def my_predict(list_texts):
    prediction = model.predict(list_texts)
    prediction = prediction.flatten()
    prediction = np.where(prediction > 0.5, 1, 0)
    return prediction

## Evaluating model using the testing dataset

In [50]:
y_predicted = my_predict(x_test)
y_predicted

12/12 [==============================] - 25s 2s/step


array([0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,

In [51]:
y_test = np.array(y_test)
y_test

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,

## Confusion Matrix

In [52]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_predicted)

array([[170,  17],
       [ 15, 172]])

## Accuracy

In [55]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predicted)

0.9144385026737968

## Testing model with custom data

In [43]:
sample_dataset = [
    'You can win a lot of money, register in the link below',
    'You have an iPhone 10, spin the image below to claim your prize and it will be delivered in your door step',
    'You have an offer, the company will give you 50% off on every item purchased.',
    "Hey Bravin, don't be late for the meeting tomorrow will start lot exactly 10:30 am",
    "See you monday, we have alot to talk about the future of this company ."
]

In [48]:
my_predict(sample_dataset)

1/1 [==============================] - 0s 429ms/step
[[0.62877375]
 [0.9148771 ]
 [0.44142482]
 [0.67397565]
 [0.11431483]]
prediction2
[0.62877375 0.9148771  0.44142482 0.67397565 0.11431483]


array([1, 1, 0, 1, 0])

## Ref:

- https://www.section.io/engineering-education/classification-model-using-bert-and-tensorflow/